# Pandas и большие файлы

In [98]:
import pandas as pd

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. 
Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. 
Под временем жизни понимается 
разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [55]:
task = pd.read_csv('ml-latest-small/ratings.csv')
task.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [56]:
second = (task.groupby('userId')
          .agg(min_ts = pd.NamedAgg(column = 'timestamp', aggfunc = min), max_ts = pd.NamedAgg(column = 'timestamp', aggfunc = max), lifetime = pd.NamedAgg(column = 'timestamp', aggfunc = lambda x: (max(x)-min(x))), count = pd.NamedAgg(column = 'movieId', aggfunc = lambda x: x.value_counts().count())))
second = second[second['count'] >= 100]
second.head()


,min_ts,max_ts,lifetime,count
userId,,,,
4,949778714,949982274,203560,204
5,1163373044,1163375145,2101,100
8,1154389340,1154474527,85187,116
15,997937239,1469330735,471393496,1700
17,1127468587,1127476640,8053,363


In [50]:
third = task.groupby('userId').agg({'timestamp': ['min', 'max', 'count']})
third.head()

timestamp                  
               min         max count
userId                              
1       1260759108  1260759205    20
2        835355395   835356246    76
3       1298861589  1298932787    51
4        949778714   949982274   204
5       1163373044  1163375145   100

# N2
Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). 
Необходимо сформировать две таблицы:
-таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
-аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [99]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [100]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [101]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [102]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [104]:
dz4 = rzd.merge(auto,on = 'client_id', how = 'outer').merge(air, on = 'client_id', how = 'outer' )
dz4

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [106]:
dz5 = dz4.merge(client_base, on = 'client_id', how = 'outer')
dz5

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,NaN,4834.0,4.0,Строителей 18
6,117,NaN,98.0,13.0,Панфиловская 33
7,118,NaN,NaN,173.0,Мастеркова 4


### Упражнение
Для каждого пользователя user_id из файла sales_db.csv посчитайте самую дорогую покупку (в столбце cost)

In [13]:
df = pd.read_csv('sales_db.csv')
df

,user_id,category,cost
0,1,Электроника,87
1,1,Дача,10
2,1,Дети,2
3,1,Дача,199
4,2,Авто,3
5,2,Авто,35
6,3,Дети,8
7,9,Дети,17


In [8]:
df.groupby(df.user_id).max()[['cost']]

,cost
user_id,
1,199
2,35
3,8
9,17


In [ ]:
previous_user = None

In [11]:
with open('sales_db.csv') as f:
    f.readline()
    for line in f:
        line = line.strip().split(',')
        
        user_id, category,cost = line
        print(user_id, category,cost)
        
        if previous_user:
            if previous_user != user_id
            ...
        else:
            previous_user = user_id

1 Электроника 87
1 Дача 10
1 Дети 2
1 Дача 199
2 Авто 3
2 Авто 35
3 Дети 8
9 Дети 17


# Объединение датафреймов

### Данные со слайдов

In [80]:
visits = pd.DataFrame(
    {
        'user_id': [11, 22, 55, 11, 77],
        'source': ['ad', 'yandex', 'email', 'google', 'ad']
    }
)
visits = visits[['user_id', 'source']]
visits

,user_id,source
0,11,ad
1,22,yandex
2,55,email
3,11,google
4,77,ad


In [81]:
purchases = pd.DataFrame(
    {
        'user_id': [11, 22, 55, 11, 99],
        'category': ['Спорт', 'Авто', 'Дача', 'Спорт', 'Авто'],
    }
)

purchases = purchases[['user_id', 'category']]
purchases

,user_id,category
0,11,Спорт
1,22,Авто
2,55,Дача
3,11,Спорт
4,99,Авто


In [82]:
visits_grouped = visits.groupby('user_id').count()
visits_grouped.rename(columns={'source': 'visits'}, inplace=True)
visits_grouped

,visits
user_id,
11,2
22,1
55,1
77,1


In [83]:
visits.groupby('user_id').count().reset_index()

,user_id,source
0,11,2
1,22,1
2,55,1
3,77,1


In [84]:
purchases_pivot = purchases.pivot_table(index='user_id', columns='category', values='user_id', 
                                        aggfunc='size', fill_value=0)
purchases_pivot

category,Авто,Дача,Спорт
user_id,,,
11,0,0,2
22,1,0,0
55,0,1,0
99,1,0,0


In [85]:
purchases_pivot.reset_index()

category,user_id,Авто,Дача,Спорт
0,11,0,0,2
1,22,1,0,0
2,55,0,1,0
3,99,1,0,0


In [86]:
visits_grouped.join(purchases_pivot)

,visits,Авто,Дача,Спорт
user_id,,,,
11,2,0.0,0.0,2.0
22,1,1.0,0.0,0.0
55,1,0.0,1.0,0.0
77,1,NaN,NaN,NaN


### LEFT join
Каждой строчке в левой таблице ищет соответствие в правой

In [22]:
visits_grouped.join(purchases_pivot, how='left')

,visits,Авто,Дача,Спорт
user_id,,,,
11,2,0.0,0.0,2.0
22,1,1.0,0.0,0.0
55,1,0.0,1.0,0.0
77,1,NaN,NaN,NaN


### RIGHT join
Каждой строчке в правой таблице ищет соответствие в левой

In [23]:
visits_grouped.join(purchases_pivot, how='right')

,visits,Авто,Дача,Спорт
user_id,,,,
11,2.0,0,0,2
22,1.0,1,0,0
55,1.0,0,1,0
99,NaN,1,0,0


### Упражнение
Дана статистика:
- ID клиентов и их имена (датафрейм clients)
- статистика доходов (earnings)
- статистика расходов (spending)

Определите имена клиентов, расходы которых превышают доходы.

In [ ]:
# подсказка - по умолчанию в методе merge объединение НЕ left join

?pd.DataFrame.merge

In [29]:
clients = pd.DataFrame(
    {
        'id': [43018, 48329, 51043, 74943, 75029],
        'name': ['Марков Илья', 'Зарицкая Елизавета', 'Благова Дарья', 'Слепова Елена', 'Гордецкий Максим'],
    }
)

clients

,id,name
0,43018,Марков Илья
1,48329,Зарицкая Елизавета
2,51043,Благова Дарья
3,74943,Слепова Елена
4,75029,Гордецкий Максим


In [30]:
earnings = pd.DataFrame(
    {
        'id': [51043, 48329, 74943, 75029, 43018],
        'debit': [34500, 12400, 89044, 5355, 19800],
    }
)

earnings

,id,debit
0,51043,34500
1,48329,12400
2,74943,89044
3,75029,5355
4,43018,19800


In [31]:
spending = pd.DataFrame(
    {
        'id': [51043, 48329, 74943, 75029, 43018],
        'credit': [22990, 2500, 69880, 6000, 29000],
    }
)

spending

,id,credit
0,51043,22990
1,48329,2500
2,74943,69880
3,75029,6000
4,43018,29000


In [33]:
data1 = clients.merge(spending, on= 'id', how = 'left')
data2 = data1.merge(earnings, on= 'id', how = 'left')
data2

,id,name,credit,debit
0,43018,Марков Илья,29000,19800
1,48329,Зарицкая Елизавета,2500,12400
2,51043,Благова Дарья,22990,34500
3,74943,Слепова Елена,69880,89044
4,75029,Гордецкий Максим,6000,5355


In [ ]:
#lsuffix, rsuffix for different names, to look up later

### INNER join
Оставляет строчки, которые есть в обеих таблицах

In [ ]:
visits_grouped.join(purchases_pivot, how='inner')

### Outer join
Оставляет все строчки

In [ ]:
visits_grouped.join(purchases_pivot, how='outer')

# Конкатенация таблиц

In [ ]:
a = pd.DataFrame({'date': ['2020-01-01', '2020-01-02', '2020-01-03'], 'value_a': [1, 2, 3]})
b = pd.DataFrame({'date': ['2020-01-01', '2020-01-02', '2020-01-03'], 'value_b': [3, 4, 5]})

In [ ]:
a

In [ ]:
b

In [ ]:
pd.concat([a, b])

In [ ]:
# объединение по горизонтали
pd.concat([a, b], axis=1)

### Дубликаты при объединении таблиц

In [34]:
ratings = pd.read_csv('ratings_example.txt', sep = '\t')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144


In [35]:
movies = pd.read_csv('movies_example.txt', sep = '\t')
movies.head()

,movieId,title,genres
0,31,Dangerous Minds (1995),Drama
1,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
2,31,Dangerous Minds (1995),Drama


In [36]:
# ¯\_(ツ)_/¯

ratings.merge(movies, how='left', on='movieId')

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,31,2.5,1260759144,Dangerous Minds (1995),Drama


In [37]:
movies.drop_duplicates(subset = 'movieId', keep = 'first', inplace = True)
movies.head()

#inplace применяет ко всему датафрейму, чтоб не писать Х=....

,movieId,title,genres
0,31,Dangerous Minds (1995),Drama
1,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller


In [38]:
ratings.merge(movies, how = 'left', on = 'movieId')

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama


In [39]:
ratings.merge(movies, how = 'right', on = 'movieId')

,userId,movieId,rating,timestamp,title,genres
0,1.0,31,2.5,1.260759e+09,Dangerous Minds (1995),Drama
1,NaN,32,NaN,NaN,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller


### Упражнение
Объедините датафреймы с визитами и покупками на сайте по ключу date. Обратите внимание, что в датафрейме визитов имеются дубликаты по дате.

In [48]:
visits = pd.DataFrame(
    {'date': ['2019-11-01', '2019-11-01', '2019-11-02', '2019-11-02', '2019-11-03'], 
     'source': ['organic', 'paid', 'organic', 'paid', 'organic'], 
     'visits': [16825, 1952, 21890, 376, 19509]}
)

visits

,date,source,visits
0,2019-11-01,organic,16825
1,2019-11-01,paid,1952
2,2019-11-02,organic,21890
3,2019-11-02,paid,376
4,2019-11-03,organic,19509


In [49]:
orders = pd.DataFrame(
    {'date': ['2019-11-01', '2019-11-02', '2019-11-03'],
     'orders': [198, 225, 201]}
)

orders

,date,orders
0,2019-11-01,198
1,2019-11-02,225
2,2019-11-03,201


In [50]:
visits.pivot_table(index='date', columns='source', aggfunc='sum', values='visits')

source,organic,paid
date,,
2019-11-01,16825.0,1952.0
2019-11-02,21890.0,376.0
2019-11-03,19509.0,NaN


In [51]:
visits.pivot_table(index='date', columns='source', aggfunc='sum', values='visits').reset_index()

source,date,organic,paid
0,2019-11-01,16825.0,1952.0
1,2019-11-02,21890.0,376.0
2,2019-11-03,19509.0,NaN


In [55]:
(
    visits
    .pivot_table(index='date', columns='source', aggfunc='sum', values='visits')
    .reset_index() #возвращает индекс для применения merge
    .merge(orders, how='left')
)

,date,organic,paid,orders
0,2019-11-01,16825.0,1952.0,198
1,2019-11-02,21890.0,376.0,225
2,2019-11-03,19509.0,NaN,201


In [43]:
visits.join(orders, how = 'outer', lsuffix='_left', rsuffix='_right')

,date_left,source,visits,date_right,orders
0,2019-11-01,organic,16825,2019-11-01,198.0
1,2019-11-01,paid,1952,2019-11-02,225.0
2,2019-11-02,organic,21890,2019-11-03,201.0
3,2019-11-02,paid,376,NaN,NaN
4,2019-11-03,organic,19509,NaN,NaN


### Оптимизация хранения данных

In [56]:
# 2.4mb
ratings = pd.read_csv('ml-latest-small/ratings.csv')

# 0.5mb
movies = pd.read_csv('ml-latest-small/movies.csv')
joined = ratings.merge(movies, how='left', on='movieId')

In [58]:
joined.head()

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama


In [61]:
movies.movieId.nunique()

9125

In [ ]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

In [ ]:
len(ratings)

In [ ]:
movies = pd.read_csv('ml-latest-small/movies.csv')
movies.head()

In [ ]:
len(movies)

In [ ]:
len(ratings) == len(joined)

In [ ]:
joined.to_csv('joined_ratings.csv', index=False)

In [ ]:
joined.head()

In [ ]:
logs = joined[['userId', 'movieId', 'rating']].head()

In [ ]:
len(joined[['movieId', 'title', 'genres']].drop_duplicates())

### Какой жанр имеет самые высокие рейтинги?

In [ ]:
import numpy as np

In [ ]:
genres = ['Drama', 'Action', 'Thriller']

In [ ]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

In [ ]:
len(ratings)

In [ ]:
movies = pd.read_csv('ml-latest-small/movies.csv')
movies.head()

In [ ]:
len(movies)

In [ ]:
joined = ratings.merge(movies, on='movieId', how='left')
joined.head()

In [ ]:
# рекомендуемая проверка на возможные дубликаты

len(ratings) == len(joined)

### Считаем рейтинг жанров

In [ ]:
# еще раз список жанров

genres = ['Drama', 'Action', 'Thriller']

In [ ]:
def genres_ratings(row):
    """Возвращает рейтинг, если он есть в списке жанров данного фильма"""
    
    return pd.Series([row['rating'] if genre in row['genres'] else np.NaN for genre in genres])

In [ ]:
%%time
joined[genres] = joined.apply(genres_ratings, axis=1)

In [ ]:
def genres_ratings_version_2(row):
    """Возвращает рейтинг, если он есть в списке жанров данного фильма"""
    
    for genre in genres:
        if genre in row.genres:
            row[genre] = row.rating
    
    return rating

In [ ]:
joined[genres] = joined.apply(genres_ratings, axis=1)
joined.head()

### Упражнение
Выведите средний рейтинг каждого жанра из списка genres

### К домашнему заданию, задача 2
Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [ ]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

In [ ]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

In [ ]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

In [ ]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base